In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from konlpy.tag import Okt

import os

from keybert import KeyBERT
import tqdm

c:\ProgramData\Anaconda3\envs\DINS\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [70]:
data = pd.read_csv('./Data/SBS.csv')

data.head(1)

,title,headline,date,link,content,category,site
0,"이란 남부 원유시설서 화재…업체 대표 ""정체불명 세력이 방화""",이란 남부 원유시설에서 불이 나 한때 생산이 중단됐다고 현지 언론이 13일 보도했습...,2022.09.13,https://news.sbs.co.kr//news/endPage.do?news_i...,이란 남부 원유시설에서 불이 나 한때 생산이 중단됐다고 현지 언론이 13일(현지...,국제,sbs


In [71]:
data.date = data.date.str.split('.').str[1]

In [73]:
data_month = data.groupby('date').get_group('10')
len(data_month)

6552

In [75]:
categories = data_month.category.unique()

len(categories)
# '사회', '국제', '스포츠', '생활·문화', '경제', '정치', '연예'

7

In [76]:
okt = Okt()

In [104]:
datas = []

for category in categories:
    try:
        datas.append(data_month.groupby('category').get_group(category).content)
    except (KeyError):
        pass

In [122]:
datas_nouns = []

for contents in datas:
    content_nouns = ''
    
    for text in contents:
        text_nouns = okt.nouns(text)
        
        for noun in text_nouns:
            content_nouns += noun + ' '
            
    datas_nouns.append(content_nouns)

In [123]:
model = KeyBERT('distilbert-base-nli-mean-tokens')

In [124]:
datas_keywords = []

for content_nouns in datas_nouns:
    datas_keywords.append(model.extract_keywords(content_nouns, keyphrase_ngram_range=(1,1), top_n=20, use_mmr=True))

In [129]:
result_keywords = []

category_idx = 0

for keywords in datas_keywords:
    for keyword in keywords:
        result_keywords.append(('10', categories[category_idx], ) + keyword) # (date, category, keyword, importance)

    category_idx += 1

In [132]:
result = pd.DataFrame(result_keywords, columns=['month', 'category', 'keyword', 'importance'])

result

,month,category,keyword,importance
0,10,스포츠,월드베이스볼클래식,0.8933
1,10,스포츠,다이아몬드백스,0.8778
2,10,스포츠,유럽챔피언스리그,0.8759
3,10,스포츠,도미니카공화국,0.8708
4,10,스포츠,사우스캐롤라이나주,0.8674
...,...,...,...,...
135,10,연예,후반작업,0.8105
136,10,연예,지방검찰청,0.8022
137,10,연예,연세대학교,0.8021
138,10,연예,블랙박스,0.7925


In [133]:
result.to_csv('./Data/keywords(month).csv', index=False)

In [139]:
import pymysql

In [140]:
keywords = pd.read_csv('./Data/keywords(month).csv')

keywords

,month,category,keyword,importance
0,10,스포츠,월드베이스볼클래식,0.8933
1,10,스포츠,다이아몬드백스,0.8778
2,10,스포츠,유럽챔피언스리그,0.8759
3,10,스포츠,도미니카공화국,0.8708
4,10,스포츠,사우스캐롤라이나주,0.8674
...,...,...,...,...
135,10,연예,후반작업,0.8105
136,10,연예,지방검찰청,0.8022
137,10,연예,연세대학교,0.8021
138,10,연예,블랙박스,0.7925


In [144]:
sql_insert = 'INSERT INTO keywords_month (month, category, keyword, importance) VALUES (%s, %s, %s, %s)'

In [145]:
conn = pymysql.connect(host='localhost', user='root', password='1234', db='dins', charset='utf8')
cursor = conn.cursor()


for index, row in keywords.iterrows():
    cursor.execute(sql_insert, (row.month, row.category, row.keyword, row.importance))

conn.commit()
conn.close()

In [106]:
# keywords = model.extract_keywords(contents_nouns, keyphrase_ngram_range=(1,1), top_n=15, use_mmr=True)

# keywords